# 소개
## 분석 목적
- 분석 목적 : 세계2차대전 공중 폭격작전이 완료되는 시점을 예측
- 과거 세계2차대전 악천 후로 인해 D-day 착륙이 거의 연기되었음을 상기하면서 폭격 작전 데이터 세트의 임무와 비교하기 위해 기상데이터와 같이 분석

## 데이터
- 작업데이터
    - 2차 세계대전 항공전 실사 시퀀스(종이 임무보고서 자료) -> 디지털화 
    - 날짜, 충돌, 지리적 위치 및 기타 데이터 요소 포함, 1939년~1945년
- 기상데이터
    - 악천후로 인해 D-day착륙이 거의 연기됨을 보고 임무와 비교하기 위해 해당 기간의 기상보고
    - 전세계 다양한 기상 관측소의 매일 기록된 기상 조건 : 강수, 강설량, 온도, 풍속, 천둥, 폭풍 등
    - 미국 국립해양대기청( https://www.kaggle.com/noaa ) 국립환경정보센터 웹사이트 참고
    - 1 번째 데이터 : 국가, 위도 및 경도와 같은 기상 관측 소 위치 포함
    - 2 번째 데이터 : 기상 관측소에서 측정된 최소, 최대 및 평균 온도를 포함

## Import packages, Load Data

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings("ignore")

#### 모듈 소개
- numpy(넘파이) : 대규모 다차원 배열을 쉽게 처리 할 수 있도록 지원하는 파이썬의 패키지
- pandas(판다스) : 쉽고 직관적인 관계형 또는 분류된 데이터로 작업 할 수 있도록 설계된 빠르고 유연하며 표현이 풍부한 데이터 구조를 제공하는 Python 패키지
- seaborn(씨본) : Matplotlib을 기반으로 다양한 색상 테마와 통계용 차트 등의 기능을 추가한 시각화 패키지
- matplotlib(맷플롯라이브러리) : 파이썬에서 매트랩(MATLAB, 수치해석 및 프로그래밍 환경을 제공하는 공학용 소프트웨어)과 유사한 그래프 표시를 가능케 하는 패키지
    - pyplot : 자동으로 figure와 axes를 생성하며, 정의된 플롯을 얻을 수 있도록 만들어 줌
- plotly : 다른 시각화에 비해 더 아름답고, 웹 상에 publish 하여 interactive visualization 용도로 사용하는데 매우 훌륭한 패키지
    - offine : 오프라인 notebook 환경에서 plotly를 사용할 수 있게 해줌
        - init_notebook_mode : jupyter notebook 모드 켜기
        - iplot : jupyter notebook 환경에서 그래프 그리기
    - graph_objs : Plotly의 모든 그래프 개체에 대한 정의를 가져옴
- os : operating system의 약자로 운영체제를 의미, 운영체제와의 상호작용을 돕는 다양한 기능을 제공, 위에서는 데이터가 있는 디렉토리 위치 확인 및 설정
- warnings : 경고를 제어하는 패키지, 위에서는 무시하게 만듦

In [ ]:
#bombing data
aerial = pd.read_csv("../input/world-war-ii/operations.csv")

#first wether data that includes location like country, latitude and longitude
weather_station_location = pd.read_csv("../input/weatherww2/Weather Station Locations.csv")

#Second weather data that includes measured min, max and mean temperatures
weather = pd.read_csv("../input/weatherww2/Summary of Weather.csv")

## Data Desscription
- Aerial bombing Data description:
    - Mission Date: Date of mission
    - Theater of Operations: Region in which active military operations are in progress; "the army was in the field awaiting action"; Example: "he served in the Vietnam theater for three years"
    - Country: Country that makes mission or operation like USA
    - Air Force: Name or id of air force unity like 5AF
    - Aircraft Series: Model or type of aircraft like B24
    - Callsign: Before bomb attack, message, code, announcement, or tune that is broadcast by radio.
    - Takeoff Base: Takeoff airport name like Ponte Olivo Airfield
    - Takeoff Location: takeoff region Sicily
    - Takeoff Latitude: Latitude of takeoff region
    - Takeoff Longitude: Longitude of takeoff region
    - Target Country: Target country like Germany
    - Target City: Target city like Berlin
    - Target Type: Type of target like city area
    - Target Industry: Target industy like town or urban
    - Target Priority: Target priority like 1 (most)
    - Target Latitude: Latitude of target
    - Target Longitude: Longitude of target
- Weather Condition data description:
- Weather station location:
    - WBAN: Weather station number
    - NAME: weather station name
    - STATE/COUNTRY ID: acronym of countries
    - Latitude: Latitude of weather station
    - Longitude: Longitude of weather station
- Weather:
    - STA: eather station number (WBAN)
    - Date: Date of temperature measurement
    - MeanTemp: Mean temperature

In [ ]:
aerial.info()

In [ ]:
weather_station_location.info()

In [ ]:
weather.info()

# 왜 데이터를 줄였을까..? 필요없다고 생각되어서? 아니면 데이터 변수에 대한 뜻을 몰라서?? 찾아봐야할 것 같다

## Data Cleaning


In [ ]:
aerial.sample(10)

# NaN은 info()에서 개수에서는 빠지는데 Null로는 체크 안 됨(왜?)
# code에서는 Null로 인식한다 뭐짘ㅋㅋㅋㅋㅋㅋ개웃기네

In [ ]:
# drop countries that are NaN
aerial = aerial[pd.isna(aerial.Country)==False]

# drop if target longitude is NaN
aerial = aerial[pd.isna(aerial['Target Longitude'])==False]

# drop if takeoff longitude is NaN
aerial = aerial[pd.isna(aerial['Takeoff Longitude'])==False]

# drop unused features
drop_list = ['Mission ID','Unit ID','Target ID','Altitude(Hundreds of Feet)']

# drop unused features
drop_list = ['Mission ID','Unit ID','Target ID','Altitude (Hundreds of Feet)','Airborne Aircraft',
             'Attacking Aircraft', 'Bombing Aircraft', 'Aircraft Returned',
             'Aircraft Failed', 'Aircraft Damaged', 'Aircraft Lost',
             'High Explosives', 'High Explosives Type','Mission Type',
             'High Explosives Weight (Pounds)', 'High Explosives Weight (Tons)',
             'Incendiary Devices', 'Incendiary Devices Type',
             'Incendiary Devices Weight (Pounds)',
             'Incendiary Devices Weight (Tons)', 'Fragmentation Devices',
             'Fragmentation Devices Type', 'Fragmentation Devices Weight (Pounds)',
             'Fragmentation Devices Weight (Tons)', 'Total Weight (Pounds)',
             'Total Weight (Tons)', 'Time Over Target', 'Bomb Damage Assessment','Source ID']

aerial.drop(drop_list, axis=1, inplace=True)
aerial = aerial[aerial.iloc[:,8]!="4248"] # drop this takeoff latitude
aerial = aerial[aerial.iloc[:,9]!=1355] # drop this takeoff longitude

# 저 변수들을 다 잘라버린 이유는 무엇일까?
# 반대로 남은 변수들의 왜 선별된 것일까?, 샘플이 너무 많이 줄었다.

In [ ]:
aerial.info()

In [ ]:
# what we will use only
weather_station_location = weather_station_location.loc[:,["WBAN","NAME","STATE/COUNTRY ID","Latitude","Longitude"]]
weather_station_location.info()

In [ ]:
# what we will use only
weather = weather.loc[:,["STA","Date","MeanTemp"]]
weather.info()

## Data Visualization
- 얼마나 많은 공격 국가가 있는지
- 목표 국가 top 10
- 전투기 top 10
- 이륙기지위치(공격국가)
- 목표 위치(If you do not understand methods of pyplot look at my pyplot tutorial: https://www.kaggle.com/kanncaa1/plotly-tutorial-for-beginners)
- 폭격 경로
- 작전 지역
- 기상 관측소 위치

In [ ]:
# country
print(aerial['Country'].value_counts())
plt.figure(figsize=(22,10))
sns.countplot(aerial["Country"])
plt.show()

In [ ]:
# Top target countries
print(aerial['Target Country'].value_counts()[:10])
plt.figure(figsize=(22,10))
sns.countplot(aerial['Target Country'])
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Aircraft Series
data = aerial['Aircraft Series'].value_counts()
print(data[:10])
data = [go.Bar(
               x=data[:10].index,
               y=data[:10].values,
               hoverinfo='text',
               marker=dict(color='rgba(177,14,22,0.5)',
                           line=dict(color='rgb(0,0,0)',width=1.5)),
)]

layout = dict(title='Aircraft Series')
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
# Attack
aerial["color"] =""
aerial.color[aerial.Country=="USA"]="rgb(0,116,217)"
aerial.color[aerial.Country=="GREAT BRITAIN"]="rgb(255,65,54)"
aerial.color[aerial.Country=="NEW ZEALAND"]="rgb(133,20,75)"
aerial.color[aerial.Country=="SOUTH AFRICA"]="rgb(255,133,27)"

data = [dict(
             type='scattergeo',
             lon=aerial['Takeoff Longitude'],
             lat=aerial['Takeoff Latitude'],
             hoverinfo='text',
             text="Country:"+aerial.Country+"takeoff Location:"+aerial["Takeoff Location"]+"Takeoff Base:"+aerial['Takeoff Base'],
             mode = 'markers',
             marker=dict(
                         sizemode='area',
                         sizeref=1,
                         size=10,
                         line=dict(width=1, color="white"),
                         color=aerial["color"],
                         opacity=0.7),
        )]
layout = dict(
              title='Countties Take Off Bases',
              hovermode='closest',
              geo=dict(showframe=False, showland=True, showcoastlines=True, showcountries=True,
                       countrywidth=1, projection=dict(type='Mercator'),
                       landcolor='rgb(217,217,217)',
                       subunitwidth=1,
                       showlakes=True,
                       lakecolor='rgb(255,255,255)',
                       countrycolor='rgb(5,5,5)')
              )
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [ ]:
#Bombing paths
#trace1
airports = [dict(type='scattergeo',
                 lon=aerial['Takeoff Longitude'],
                 lat=aerial['Takeoff Latitude'],
                 hoverinfo='text',
                 text="Country:"+aerial.Country+"Takeoff Location:"+aerial["Takeoff Location"]+"Takeoff Base:"+aerial["Takeoff Base"],
                 mode='markers',
                 marker=dict(size=5,
                             color=aerial["color"],
                             line=dict(width=1,
                                       color="white"
                                      )
                            )
                 )
            ]

#trace2
targets = [dict(type='scattergeo',
                lon=aerial['Target Longitude'],
                lat=aerial['Target Latitude'],
                hoverinfo='text',
                text="Target Country: "+aerial["Target Country"]+" Target City: "+aerial["Target City"],
                mode='markers',
                marker=dict(size=1,
                            color="red",
                            line=dict(width=0.4,
                                      color="red"
                                      )
                           )
               )
          ]

#trace3
flight_paths=[]
for i in range(len(aerial['Target Longitude'])):
    flight_paths.append(
                        dict(
                             type='scattergeo',
                             lon=[aerial.iloc[i,9], aerial.iloc[i,16]],
                             lat=[aerial.iloc[i,8], aerial.iloc[i,15]],
                             mode='lines',
                             line=dict(width=0.7,
                                       color='black'
                             ),
                         opacity=0.6
                         ),
     )

layout=dict(
            title='Bombing Paths from attacker country to Target',
            hovermode='closest',
            geo=dict(showframe=False, showland=True, showcoastlines=True, showcountries=True,
                     countrywidth=1, projection=dict(type='Mercator'),
                     subunitwidth=1,
                     showlakes=True,
                     lakecolor='rgb(255,255,255)',
                     countrycolor='rgb(5,5,5)')
)

fig=dict(data=flight_paths + airports + targets, layout=layout)
iplot(fig)

위의 지도를 보았을 때 대부분의 폭격이 이루어진 장소 : 지중해
- ETO : 유럽 작전 지역
- PTO : 태평양 작전 지역
- MTO : 지중해 작전 지역
- CBI : 중국-버바-인도 작전지역
- EAST AFICA : 동아프리카 작전 지
<a href="http://imgbb.com/"><img src="http://image.ibb.co/bYvFzx/mto.png" alt="mto" border="0"></a>

In [ ]:
#Theater of Operations
print(aerial['Theater of Operations'].value_counts())
plt.figure(figsize=(22,10))
sns.countplot(aerial['Theater of Operations'])
plt.show()

In [ ]:
# Weather station locations
data = [dict(type='scattergeo',
             lon=weather_station_location.Longitude,
             lat=weather_station_location.Latitude,
             hoverinfo='text',
             text="Name:"+weather_station_location.NAME+"Country:"+weather_station_location["STATE/COUNTRY ID"],
             mode='markers',
             marker=dict(sizemode='area',
                         sizeref=1,
                         size=8,
                         line=dict(width=1,color="white"),
                         color="blue",
                         opacity=0.7),
            )
       ]
layout = dict(title="Weather Station Locations",
              hovermode='closest',
              geo=dict(showframe=False, showland=True, showcoastlines=True, showcountries=True,
                       countrywidth=1, projection=dict(type='Mercator'),
                       landcolor='rgb(217,217,217)',
                       subunitwidth=1,
                       showlakes=True,
                       lakecolor='rgb(255,255,255)',
                       countrycolor='rgb(5,5,5)')
             )
fig = go.Figure(data=data,layout=layout)
iplot(fig)

USA와 BURMA 전쟁
- USA가 BURMA에 1942~1945년까지 폭격
- 가장 가까운 기상관측소는 BINDUKURI(1943~1945 데이터)
- 날짜 기능을 날짜 시간 객체화

In [ ]:
weather_station_id = weather_station_location[weather_station_location.NAME=="BIRNDUKURI"].WBAN
weather_bin = weather[weather.STA==32907]
weather_bin["Date"] = pd.to_datetime(weather_bin["Date"])
plt.figure(figsize=(22,10))
plt.plot(weather_bin.Date,weather_bin.MeanTemp)
plt.title("Mean Temperature of Bindukuri Area")
plt.xlabel("Date")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.show()

기온
- 12도에서 32도 사이
- 겨울엔 낮고 여름엔 높음
####  **계절별로 평균을 봐야하나...?**

In [ ]:
aerial = pd.read_csv("../input/world-war-ii/operations.csv")
aerial["year"] = [each.split("/")[2] for each in aerial["Mission Date"]]
aerial["month"] = [each.split("/")[0] for each in aerial["Mission Date"]]
aerial = aerial[aerial["year"]>="1943"]
aerial = aerial[aerial["month"]>="8"]

aerial["Mission Date"] = pd.to_datetime(aerial["Mission Date"])

attack = "USA"
target = "BURMA"
city = "KATHA"

aerial_war = aerial[aerial.Country==attack]
aerial_war = aerial_war[aerial_war["Target Country"]==target]
aerial_war = aerial_war[aerial_war["Target City"]==city]

In [ ]:
# 더 좋은 dummy 코드가 있으면 수정하라고 적혀있음
liste = []
aa = []
for each in aerial_war["Mission Date"] :
    dummy = weather_bin[weather_bin.Date==each]
    liste.append(dummy["MeanTemp"].values)
aerial_war["dene"] = liste
for each in aerial_war.dene.values :
    aa.append(each[0])

# Create a trace
trace = go.Scatter(x=weather_bin.Date,
                   mode="lines",
                   y=weather_bin.MeanTemp,
                   marker=dict(color='rgba(16,112,2,0.8)'), #rgba는 색 + 투명도
                   name = "Mean Temperature")
trace1 = go.Scatter(x=aerial_war["Mission Date"],
                    mode='markers',
                    y=aa,
                    marker=dict(color='rgba(16,0,200,1)'),
                    name="bombing temperature")
layout = dict(title='Mean Temperature ---Bombing Date and Mean Temperature at this Date')
data = [trace,trace1]

fig = dict(data=data, layout=layout)
iplot(fig)

- 녹색 선은 Bindukuri의 평균 기온
- 파란 점은 폭격 날의 온도
- 그래프 상으로 겨울보단 여름에 폭격을 진행

## 시계열 분석 (ARIMA)
- 시계열이란?
    - 일정 시간 간격으로 수집되는 데이터
    - 대부분의 시계열 데이터는 추세가 있음(이 데이터는 계절성 추)
- 시계열의 정상성 가정
    - 정지된 시계열인지 아닌지 확인하는 가정
        - 상수평균
        - 일정한 분산
        - 시간에 의존하지 않는 자기공분(시계열과 지연시계열간의 공분산)
    - 시계열의 계절성 시계열을 시각화하고 확인
    
#### **더 확인해보고 설명이 추가로 필요할 것**
- 왜 ARIMA로 시작했는지
- 정상성에 대한 설명
- 정상성 조건
- 그 외에 필요한 시계열의 가정

In [ ]:
# Mean temperature of Bindikuri area
plt.figure(figsize=(22,10))
plt.plot(weather_bin.Date,weather_bin.MeanTemp)
plt.title("Mean temperature of Bindikuri Area")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.show()

#lets create time series from weather
timeSeries = weather_bin.loc[:,["Date","MeanTemp"]]
timeSeries.index = timeSeries.Date
ts = timeSeries.drop("Date",axis=1)

- 시계열의 정상성 확인
     - Plotting Rolling Statistics: 윈도우 크기 6, 정지 상태를 확인하기 위해 롤링 평균과 분산 확인
     * Dickey-Fuller 테스트: 테스트 결과는 차이 신뢰 수준에 대한 **p-value** 및 일부 **임계값**으로 구성, **p-value**가 **임계값**보다 작으면 시계열은 정상적

In [ ]:
# adfuller library
from statsmodels.tsa.stattools import adfuller

# check_adfuller
def check_adfuller(ts):
    #Dickey-Fuller test
    result = adfuller(ts,autolag='AIC')
    print('Test statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:', result[4])

#check_mean_std
def check_mean_std(ts):
    # Rolling statistics
    rolmean = pd.rolling_mean(ts, window=6)
    rolstd = pd.rolling_std(ts, window=6)
    plt.figure(figsize=(22,10))
    orig = plt.plot(ts, color='red', label='Original')
    mean = plt.plot(rolmean, color='black', label='Rolling Mean')
    std = plt.plot(rolstd, color='green', label = 'Rolling Std')
    plt.xlabel("Date")
    plt.ylabel("Mean Temperature")
    plt.title('Rolling Mean & Standard Deviation')
    plt.legend()
    plt.show()
    
# check stationary : mean, variance(std ^2) and adfuller test
check_mean_std(ts)
check_adfuller(ts.MeanTemp)

- 정상성 기준
    - 상수 평균 : Black line이 일정한 값이 아니기에 정상성X
    - 상수 분산 : Green line이 일정하게 있어서 정상성O
    - p-value가 Critical Value보다 낮으면 정상성 : 정상성X
- 위 3가지 기준으로 볼 때 정상성 X으로 판정 -> 정상시계열로 만들어줘야 함

- 정상시계열화
    - 비정상성의 원인
        - 경향(Trend) : 시간의 경과에 따라 변화하는 평균, 상수평균이 일정한 값이어야 함
            - 해결법 : 이동평균 - 과거 n(window size)샘플에 대한 평균을 취함
        - 계절성(Seasonality) : 특정 시간의 변동, 시계열 정상에 대한 일정한 변동이 필요


In [ ]:
# Moving average method
window_size = 6
moving_avg = pd.rolling_mean(ts, window_size)
plt.figure(figsize=(22,10))
plt.plot(ts, color="red", label="Original")
plt.plot(moving_avg, color='black', label="moving_avg_mean")
plt.title("Mean Temperature of Bindikuri Area")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.legend()
plt.show()

In [ ]:
ts_moving_avg_diff = ts - moving_avg
ts_moving_avg_diff.dropna(inplace=True) # first 6 is nan value due to window size

# check stationary : mean, variance(std ^2) and adfuller test
check_mean_std(ts_moving_avg_diff)
check_adfuller(ts_moving_avg_diff.MeanTemp)

- 상수 평균 기준 : Black line, 일정
- 상수 분산 기준 : Green line, 일정
- p-value : Critical values 절대값 1% 기준보다 작음
- 정상시계열 달성
    - 감별법 : 보편적인 방법, 시계열과 이동 시계열의 차이를 취함

In [ ]:
# differencing method
ts_diff = ts - ts.shift()
plt.figure(figsize=(22,10))
plt.plot(ts_diff)
plt.title("Differencing method")
plt.xlabel("Date")
plt.ylabel("Differencing Mean Temperature")
plt.show()

In [ ]:
ts_diff.dropna(inplace=True) # due to shifting there is nan values
# check stationary : mean, variance(std ^2) and adfuller test
check_mean_std(ts_diff)
check_adfuller(ts_diff.MeanTemp)

- 상수 평균 기준: Black line, 일정
- 상수 분산 기준 : Green line, 일정
- p-value : Critical values 기준 1%보다 작다...? 
#### **Critical value 기준 확인해봐야한다**

## 시계열 예측
- forecationg : 차분의 결과인 ts_diff 시계열을 사용, ARIMA 사용
    - AR(자동회귀, p) : 종속 변수의 시차
        - p를 3으로 가정하면 x(t-1), x(t-2), x(t-3)을 사용하여 t(x) 예측
    - I(적분, d) : 비계절성 차이의 수
        - 여기서는 1차 차액을 취함, d=0
    - MA(이동평균, q) : 예측 방정식의 지연 예측 오차
    - (p,d,q)는 ARIMA 모델의 매개변수
    - 매개변수 선택법
        - 자기 상관 함수(ACF) : 시계열과 시계열의 지연 버전 간의 상관관계 측정
        - 부분 자기 상관 함수(PACF) : 시게열과 시계열의 시차 버전 사이의 상관관계를 측정, 중간 비교에 의해 이미 설명된 변동을 제거 후 측정

In [ ]:
# ACF and PACF
from statsmodels.tsa.stattools import acf, pacf
lag_acf = acf(ts_diff, nlags=20)
lag_pacf = pacf(ts_diff, nlags=20, method='ols')

# ACF
plt.figure(figsize=(22,10))
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff)), linestyle='--', color='gray')
plt.title('Autocorrelation Function')

# PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff)), linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

- 각 그래프의 두 점선은 신뢰구간(임계값)
    - q 선택 : ACF 차트가 처음으로 상위 신뢰구간과 교차하는 지점(1)
    - p 선택 : PACF 차트가 처음으로 상위 신뢰구간과 교차하는 지점 (1)
- 따라서 ARIMA모델의 매개변수는 (1,0,1)
     - ARIMA: statsmodels 라이브러리
     - datetime: 우리는 예측 메소드의 시작 및 종료 인덱스를 사용

In [ ]:
# ARIMA Library
from statsmodels.tsa.arima_model import ARIMA
from pandas import datetime

# Fit model
model = ARIMA(ts, order=(1,0,1)) #(ARMA) = (1,0,1)...?
model_fit = model.fit(disp=0)

# Predict
start_index = datetime(1944, 6, 25)
end_index = datetime(1945, 5, 31)
forecast = model_fit.predict(start=start_index, end=end_index)

# Visualization
plt.figure(figsize=(22,10))
plt.plot(weather_bin.Date, weather_bin.MeanTemp, label="original")
plt.plot(forecast, label="predicted")
plt.title("Time Series Forecast")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.legend()
plt.show()

- 1944년도 7월부터 예측하여 실제값과 비교
- 비슷한 결과를 보임

In [ ]:
# Predict all path
from sklearn.metrics import mean_squared_error

# Fit model
model2 = ARIMA(ts, order=(1,0,1)) # ARMA = (1,0,1)
model_fit2 = model2.fit(disp=0)
forecast2 = model_fit2.predict()
error = mean_squared_error(ts, forecast2)
print("error:", error)

# Visualization
plt.figure(figsize=(22,10))
plt.plot(weather_bin.Date, weather_bin.MeanTemp, label='original')
plt.title("Time series Forecast")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.legend()
plt.show()
# plt.savefig('graph.png')

## 결론
- 이 튜토리얼 목적 
    - 시계열 시각화 + ARIMA에 대한 튜토리얼 생성 
    - pyplot으로 맵 플롯 생성 방법
    - 시계열 예측 방법